In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hri_tools import HumorDataset
from hri_tools import SUPPORTED_DATASETS

In [3]:
print(SUPPORTED_DATASETS)

['pun_of_the_day', 'one_liners', 'reddit_jokes_last_laught', 'short_jokes', 'funlines', 'human_microedit']


In [4]:
all_data = list()
for name in SUPPORTED_DATASETS:
    all_data.append(
        HumorDataset(
            name=name
        )
    )

for i in range(len(all_data)):
    all_data[i].load()

In [5]:
for i in range(len(all_data)):
    all_data[i].calc_statistics()

for i in range(len(all_data)):
    all_data[i].print_statistics_report()


        Statistics report for pun_of_the_day dataset:
        Number of samples: 4826, pos - 2423, neg - 2403.
        Mean word length is 13.515748031496063
        Mean length by symbols is 66.64981351015334 (without space is 54.13966017405719)
        

        Statistics report for one_liners dataset:
        Number of samples: 32000, pos - 16000, neg - 16000.
        Mean word length is 12.58109375
        Mean length by symbols is 58.6513125 (without space is 48.99703125)
        

        Statistics report for reddit_jokes_last_laught dataset:
        Number of samples: 15910, pos - 2026, neg - 13884.
        Mean word length is 48.76279069767442
        Mean length by symbols is 212.84701445631677 (without space is 174.5742300439975)
        

        Statistics report for short_jokes dataset:
        Number of samples: 463315, pos - 231658, neg - 231657.
        Mean word length is 46.093908032332216
        Mean length by symbols is 232.2224598815061 (without space is 194.80

In [6]:
for i in range(len(all_data)):
    all_data[i].print_positive_sample()


        Example from "pun_of_the_day Dataset"
        Humorous eaxmple: my brother wishes he could compose smutty verse as good as mine is this scribbling ribaldry
        

        Example from "one_liners Dataset"
        Humorous eaxmple: I have this nagging fear that everyone is out to make me paranoid.
        

        Example from "reddit_jokes_last_laught Dataset"
        Humorous eaxmple: My girlfriend is a pornstar She is going to be very pissed when she finds out.
        

        Example from "short_jokes Dataset"
        Humorous eaxmple: Why do male midgets laugh so hard when they play football ? It's because the grass tickles their balls.
        

        Example from "funlines Dataset"
        Humorous eaxmple: Reporters claim new blindfolds will hinder impeachment coverage
        

        Example from "human_microedit Dataset"
        Humorous eaxmple: New WH comms director Anthony Scaramucci deletes old braincells contradicting Trump
        


In [4]:
%env HRI_URL=https://drive.google.com/file/d/
%env HRI_PASSWORD=

env: HRI_URL=https://drive.google.com/file/d/
env: HRI_PASSWORD=


In [1]:
from hri_tools import downlaod

In [ ]:
downlaod()